In [1]:
import dgl
from dgl.data import CiteseerGraphDataset, CoraGraphDataset, PubmedGraphDataset
from dgl import AddSelfLoop
from dgl.dataloading import GraphDataLoader
from torch.utils.data import DataLoader
from dgl.nn import EdgeGATConv, GraphConv
import torch.nn as nn
import torch
from tqdm import tqdm
from sklearn.metrics import f1_score

In [2]:
dataset = dgl.data.CSVDataset('../data/dataset/')

Done loading data from cached files.


In [4]:
train_dataset, val_datasetloader = dgl.data.utils.split_dataset(dataset, shuffle=True, frac_list=[0.9, 0.1])

In [5]:
data_loader = GraphDataLoader(dataset, batch_size=32, shuffle=True)
train_loader = GraphDataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = GraphDataLoader(val_datasetloader, batch_size=32, shuffle=True)

In [156]:
graphs = dgl.unbatch(next(iter(data_loader))[0])
len(graphs)

32

In [115]:
def collate(samples):
    # The input `samples` is a list of pairs
    #  (graph, label).
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    return batched_graph, torch.tensor(labels)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate)

In [125]:
next(iter(data_loader))[0]
dgl.batch([dataset[0][0], dataset[1][0]])

Graph(num_nodes=9, num_edges=12,
      ndata_schemes={'feat': Scheme(shape=(768,), dtype=torch.float32)}
      edata_schemes={'feat': Scheme(shape=(768,), dtype=torch.float32)})

In [167]:
g = next(iter(data_loader))[0]

# g = dgl.add_self_loop(g)
# print(dgl.unbatch(g))
print(g.ndata['feat'].shape)
layer1 = EdgeGATConv(768, 768, 15, 2, allow_zero_in_degree=True)
layer2 = EdgeGATConv(15*2, 768, 15, 2, allow_zero_in_degree=True)
# layer1 = GraphConv(768, 15, allow_zero_in_degree=True)
# layer2 = GraphConv(15, 2, allow_zero_in_degree=True)
lin_layer = nn.Linear(15 * 2, 1)
edge_feat = g.edata['feat']
node_feat = g.ndata['feat']
print(edge_feat.shape)
out1 = layer1(g, node_feat, edge_feat, get_attention=False)
print(out1.shape)
print(out1.view(g.ndata['feat'].shape[0], 30).shape)
out2 = F.relu(layer2(g, out1.view(g.ndata['feat'].shape[0], 30), edge_feat))
print(out2.shape)
out2 = out2.view(g.ndata['feat'].shape[0], 30)
print(out2.shape)
# with g.local_scope():
#     g.ndata['feat'] = out2
#     hg1 = dgl.readout_nodes(g, 'feat')
#     print(hg1.shape)
g.ndata['feat'] = out2
hg = dgl.mean_nodes(g, 'feat')
print(hg.shape)
# he = dgl.mean_edges(g, 'feat')
# print(he.shape)
out3 = lin_layer(hg)
print(out3.shape)

# out1 = F.relu(layer1(g, g.ndata['feat']))
# print(out1.shape)
# out2 = F.relu(layer2(g, out1))
# print(out2.shape)
# g.ndata['h'] = out2
# # dgl.unbatch(g)
# dgl.mean_nodes(g, 'h').shape

torch.Size([128, 768])
torch.Size([164, 768])
torch.Size([128, 2, 15])
torch.Size([128, 30])
torch.Size([128, 2, 15])
torch.Size([128, 30])
torch.Size([32, 30])
torch.Size([32, 1])


In [7]:
import dgl.nn.pytorch as dglnn
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self, in_feats, edge_feats, out_feats, num_heads):
        super(Classifier, self).__init__()
        self.in_feats = in_feats
        self.edge_feats = edge_feats
        self.out_feats = out_feats
        self.num_heads = num_heads
        self.conv1 = dglnn.EdgeGATConv(in_feats, edge_feats, out_feats, num_heads, allow_zero_in_degree=True)
        # self.conv2 = dglnn.EdgeGATConv(out_feats * num_heads, edge_feats, out_feats, num_heads, allow_zero_in_degree=True)
        self.classify = nn.Linear(out_feats * num_heads, 1)

    def forward(self, g, node_feat, edge_feat):
        h = F.relu(self.conv1(g, node_feat, edge_feat))        
        h = h.view(h.shape[0], self.out_feats * self.num_heads)
        # h = F.relu(self.conv2(g, h, edge_feat))
        # h = h.view(h.shape[0], self.out_feats * self.num_heads)
        with g.local_scope():
            g.ndata['feat'] = h
            # Calculate graph representation by average readout.
            hg = dgl.mean_nodes(g, 'feat')
            # he = dgl.mean_edges(g, 'feat')
            # hg = torch.cat([hg, he], dim=-1)``
            return torch.sigmoid(self.classify(hg))
        

def f1_metric(logits, labels):
    preds = (logits > 0.5).float()
    return f1_score(labels.cpu().numpy(), preds.cpu().numpy())

In [8]:
in_feats_dim = 768
edge_feats_dim = 768
out_feats = 15
num_heads = 1

model = Classifier(in_feats_dim, edge_feats_dim, out_feats, num_heads)
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCELoss()
# лучшую модельку созхранять, отображать метрику ф1, графики, всякие изменеия лр
best_val_loss = float('inf')
best_model = None

epoch_losses = []
val_losses = []
num_epochs = 2
for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0
    epoch_f1 = 0
    model.train()
    for i, (batched_graph, labels) in enumerate(train_loader):
        # batched_graph = dgl.add_self_loop(batched_graph)
        node_feats = batched_graph.ndata['feat']
        edge_feats = batched_graph.edata['feat']
        logits = model(batched_graph, node_feats, edge_feats)
        loss = loss_func(logits, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        epoch_loss += loss.detach().item()
        f1 = f1_metric(logits, labels)
        epoch_f1 += f1
    epoch_loss /= (i + 1)
    epoch_f1 /= (i + 1)
    print('Epoch {}, loss {:.4f}'.format(epoch, epoch_loss))
    epoch_losses.append(epoch_loss)
    
    # Validation phase
    model.eval()
    val_loss = 0
    val_f1 = 0
    val_predictions = []
    val_targets = []
    with torch.no_grad():
        for i, (batched_graph, labels) in enumerate(val_loader):
            node_feats = batched_graph.ndata['feat']
            edge_feats = batched_graph.edata['feat']
            logits = model(batched_graph, node_feats, edge_feats)
            loss = loss_func(logits, labels)
            val_loss += loss.item()
            f1 = f1_metric(logits, labels)
            val_f1 += f1
            # Convert logits to predictions
            predictions = (logits > 0.5).float()
            val_predictions.extend(predictions.tolist())
            val_targets.extend(labels.tolist())

    val_loss /= (i + 1)
    val_losses.append(val_loss)
    print(f'Epoch {epoch + 1}/{num_epochs}, '
          f'Train Loss: {epoch_loss:.4f}, '
          f'Val Loss: {val_loss:.4f}, '
          f'Val F1: {val_f1:.4f}')
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model.state_dict('GAT_model.pth')

# After training completes, load the best model state dict
model.load_state_dict(best_model)

# Save the best model
torch.save(model.state_dict(), 'GAT_model.pth')

  0%|          | 0/2 [00:00<?, ?it/s]/home/b_gainitdinov/anaconda3/envs/mncmu_ganitdinov/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/b_gainitdinov/anaconda3/envs/mncmu_ganitdinov/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/b_gainitdinov/anaconda3/envs/mncmu_ganitdinov/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parame

KeyboardInterrupt: 